In [2]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:

# ---------------- Lectura y guardado de los documentos en un data frame de pandas.


In [4]:
# -----------------------------------------------------------------------------
# create_new_file: 0 para usar un archivo existente si es posible
# o 1 para crear de nuevo
# balance_data: 0 para retornar dataframe sin balancear o 1 para retornar
# dataframe balanceado.
# lematizacion: lematiza los textos usando nltk 0 para no lematizar y 1
# para retornar el texto lematizado.
# training: une al dataset los datos etiquetados por los modelos, 1 para unir
# y 0 para usar solo datos certificados.
# -----------------------------------------------------------------------------

def get_feelings_df(create_new_file, balance_data, lematizacion, training):
    from gensim.utils import any2unicode as unicode
    from nltk.stem import SnowballStemmer
    from sklearn.utils import resample
    import os, re, sys, pandas, unidecode

    feelings_folder = '../datos sentimientos'
    feelings_df_path = feelings_folder + '/dataset listo/sentiments_full_df.csv'
    feelings_df = pandas.DataFrame(columns=['text', 'lang', 'sentiment'])

    if create_new_file == 0 and os.path.exists(feelings_df_path) == True:
        print('Cargando datos')
        feelings_df = pandas.read_csv(feelings_df_path, encoding='utf8', dtype=str, engine='python')

    elif create_new_file == 1 or os.path.exists(feelings_df_path) == False:
        path, subfolders, files_list = list(os.walk(feelings_folder))[0]
        files_list.sort()
        for i in range(len(files_list)):
            sys.stdout.write(
                "\rPreparando dataframe " + str(round(((i + 1) / (len(files_list))) * 100, 2)) + "%"
            )
            sys.stdout.flush()
            file_name, file_ext = files_list[i].split(".")

            if file_ext == 'csv':
                file_path = path + "/" + file_name + "." + file_ext
                df = pandas.read_csv(file_path, encoding='utf8', dtype=str, engine='python')
                numero_de_archivo = int(file_name.split("_")[0])

                if numero_de_archivo == 1:
                    df = df.filter(['airline_sentiment', 'text'])
                    df.columns = ['sentiment', 'text']
                    df['lang'] = 'en'
                    df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                    feelings_df = feelings_df.append(df)

                if numero_de_archivo == 2 or numero_de_archivo == 3 or numero_de_archivo == 4:
                    df = df.filter(['sentiment', 'text'])
                    df['lang'] = 'en'
                    df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                    feelings_df = feelings_df.append(df)

                if numero_de_archivo == 5 or numero_de_archivo == 6 or numero_de_archivo == 7:
                    df = df.filter(['polarity', 'text'])
                    df.columns = ['sentiment', 'text']
                    if numero_de_archivo in [5, 6]: df['lang'] = 'es'
                    if numero_de_archivo == 7: df['lang'] = 'en'
                    df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                    feelings_df = feelings_df.append(df)

                if numero_de_archivo == 8:
                    df = df.filter(['sentiment', 'text'])
                    df.columns = ['sentiment', 'text']
                    df['lang'] = 'es'
                    df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                    feelings_df = feelings_df.append(df)

        del df
        print("")
        feelings_df.reset_index(drop=True, inplace=True)

        for i, row in feelings_df.iterrows():
            sys.stdout.write(
                "\rNormalizacion de datos completada al " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) +
                "%"
            )
            sys.stdout.flush()
            feelings_df.at[i, 'text'] = (
                re.sub(' +', ' ', re.sub("http\S+", "", re.sub('\s+', ' ', str(feelings_df.at[i, 'text']))))
            ).strip()

        print("")

    feelings_df.drop_duplicates(subset=['text'], inplace=True)
    feelings_df = feelings_df.loc[feelings_df['sentiment'].isin(['0', '1', '2'])]
    feelings_df = feelings_df.loc[feelings_df['lang'].isin(['es', 'en'])]
    feelings_df.reset_index(drop=True, inplace=True)
    feelings_df.to_csv(feelings_df_path, index=False, encoding="utf-8")

    if training == 1 and os.path.exists('./dataset etiquetado modelos/taged_tweets_sample.csv'):
        print("Incrementando datos con dataset etiquetado por modelos")
        df = pandas.read_csv('./dataset etiquetado modelos/taged_tweets_sample.csv',
                             encoding='utf8', dtype=str, engine='python')
        df = df.filter(['text', 'lang', 'sentiment'])
        feelings_df = feelings_df.append(df)
        feelings_df.reset_index(drop=True, inplace=True)
        del df

    if lematizacion == 1:
        stemmer_en = SnowballStemmer('english')
        stemmer_es = SnowballStemmer('spanish')

        for i, row in feelings_df.iterrows():
            sys.stdout.write(
                "\rLematizando " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%"
            )
            sys.stdout.flush()

            if feelings_df.at[i, 'text'] is str and feelings_df.at[i, 'lang'] == 'es':
                feelings_df.at[i, 'text'] = stemmer_es.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )

            elif feelings_df.at[i, 'text'] is str and feelings_df.at[i, 'lang'] == 'en':
                feelings_df.at[i, 'text'] = stemmer_en.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )

        print("\nLematizacion finalizada")

    if balance_data == 1:
        print("Balanceando datos")
        feelings_df["Sello"] = 0

        for i, row in feelings_df.iterrows():
            sys.stdout.write(
                "\rCreando sellos de balanceamiento " +
                str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2))
                + "%"
            )
            sys.stdout.flush()
            feelings_df.at[i, 'sello'] = str(feelings_df.at[i, 'lang']) + '_' + str(feelings_df.at[i, 'sentiment'])

        min_len1 = int(min(feelings_df['sello'].value_counts()))

        print("")
        print("Balanceando")

        df_0 = resample(feelings_df[feelings_df.sello == 'en_0'], replace=False, n_samples=min_len1, random_state=1)
        df_1 = resample(feelings_df[feelings_df.sello == 'en_1'], replace=False, n_samples=min_len1, random_state=1)
        df_2 = resample(feelings_df[feelings_df.sello == 'en_2'], replace=False, n_samples=min_len1, random_state=1)
        df_3 = resample(feelings_df[feelings_df.sello == 'es_0'], replace=False, n_samples=min_len1, random_state=1)
        df_4 = resample(feelings_df[feelings_df.sello == 'es_1'], replace=False, n_samples=min_len1, random_state=1)
        df_5 = resample(feelings_df[feelings_df.sello == 'es_2'], replace=False, n_samples=min_len1, random_state=1)

        feelings_df = pandas.concat([df_0, df_1, df_2, df_3, df_4, df_5])

        print("Eliminando sellos de balanceamiento")

        feelings_df = feelings_df.filter(['text', 'lang', 'sentiment'])
        print("Datos entregados")
        feelings_df.reset_index(drop=True, inplace=True)
        return feelings_df

    elif balance_data == 0:
        print("Datos entregados")
        feelings_df.reset_index(drop=True, inplace=True)
        return feelings_df


In [5]:
df = get_feelings_df(0,1,1,0)

unable to import 'smart_open.gcs', disabling that module


Cargando datos
Lematizando 9.3%%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Lematizando 25.26%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creando sellos de balanceamiento 72.21%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Creando sellos de balanceamiento 90.51%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
print("\nTweets en el dataframe original: " + str(df.shape[0]))


Tweets en el dataframe original: 15342


In [19]:
df.head()

,text,lang,sentiment
0,@JetBlue great flight,en,0
1,@united Sivi Stewart at Lax was fantastic toni...,en,0
2,@AmericanAir 1138 got us to LGA safely. Thanks...,en,0
3,@united I should be fine. They automatically c...,en,0
4,In view of shortage of Personal Protective Equ...,en,0


In [20]:
#----------------- Lexicones
#Lectura de SentiWordNet Obtenido de 
#https://www.nltk.org/_modules/nltk/corpus/reader/sentiwordnet.html

# SentiWordNet[word] = {POS,	ID,	PosScore,	NegScore}
contador = 0
SentiWordNet = dict()
for lines in open('Data Lexicon/SentiWordNet_3.0.0.txt'):
    if lines.startswith('#'):
        continue
    line = lines.split('\t')
    palabra = line[4].split('#')[0]
    if (palabra in SentiWordNet) or (palabra==''):
        continue
    else:
        SentiWordNet[palabra]={'POS': line[0], 'ID': line[1], 'PosScore': line[2], 'NegScore': line[3]}

In [12]:
# AFFIN[word] = sentiment
AFFIN = dict()
for lines in open('Data Lexicon/AFFIN-111.txt'):
    AFFIN[lines.split('\t')[0]]=(lines.split('\t')[1]).split('\n')[0]

In [13]:
tt = TweetTokenizer()
def Lexicon(data):
    addlex = list()
    for frase in data:
        splited = tt.tokenize(frase)
        sum_swn_neg = 0
        sum_swn_pos = 0
        affin = 0
        stnet = 0
        word_stat = 0
        for word in splited:
            if word in SentiWordNet.keys():
                sum_swn_neg += float(SentiWordNet[word]['NegScore'])
                sum_swn_pos += float(SentiWordNet[word]['PosScore'])
            if word in AFFIN.keys():
                affin += float(AFFIN[word])
            
        addlex.append([sum_swn_neg, sum_swn_pos, affin])
    return addlex

In [14]:
def extension(matriz, data):
    extendLex = np.array(Lexicon(data))
    return np.append(matriz, extendLex, 1)

In [15]:
vectorizer = TfidfVectorizer(max_features=3000, min_df=7, max_df=0.8)

In [21]:
features = vectorizer.fit_transform(df['text']).toarray()

In [22]:
labels = df['sentiment']

In [23]:
vectorizacionLexicon = extension(features, df['text'])

In [25]:
#for i in vectorizacionLexicon: print(i)

In [26]:
data_train, data_test, label_train, label_test = train_test_split(vectorizacionLexicon,labels)

### Metricas obtenidas usando AdaBoostClassifier con 6 modelos y una profundidad maxima de 8

In [38]:
modelo = AdaBoostClassifier(DecisionTreeClassifier(max_depth=200), n_estimators=10)

In [39]:
modelo.fit(data_train, label_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=200,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                        

In [40]:
# ---------------- implementacion del modelo.
predictions0 = modelo.predict(data_test)

In [41]:

# ---------------- resultados del modelo.
datos_totales_prueba = len(label_test)
print("Numero de datos para la prueba: " + str(datos_totales_prueba))


Numero de datos para la prueba: 3836


In [42]:
matrix = confusion_matrix(label_test, predictions0)
aciertos = sum([matrix[i][i] for i in range(matrix.shape[0])])
precision = aciertos / datos_totales_prueba
print("Numero de aciertos totales: " + str(aciertos))
print("Precision total: " + str(precision))

Numero de aciertos totales: 2954
Precision total: 0.7700729927007299


In [43]:

def imprimir_metricas_matriz_confusion_multiclase(matrix, headers, total_datos):
    print("Numero de datos para la prueba: " + str(total_datos))
    metrics = np.zeros([len(headers), 2])
    for i in range(matrix.shape[0]):
        precision = matrix[i][i] / sum(matrix[i])
        recoil = matrix[i][i] / sum([matrix[y][i] for y in range(matrix.shape[0])])
        metrics[i][0] = precision
        metrics[i][-1] = recoil
    precision_promedio = sum([metrics[y][0] for y in range(metrics.shape[0])]) / metrics.shape[0]
    recall_promedio = sum([metrics[y][-1] for y in range(metrics.shape[0])]) / metrics.shape[0]
    print("Precision promedio: " + str(precision_promedio))
    print("Recall promedio: " + str(recall_promedio))
    print("\nMatriz de confusion: \n" + str(tabulate(
        matrix,
        headers=headers,
        showindex=headers,
        tablefmt='grid')
    ))
    print("\nMetricas de desempeño: \n" + str(tabulate(
        metrics,
        headers=['precision', 'recall'],
        showindex=headers,
        tablefmt='grid')
    ))
    print("")


imprimir_metricas_matriz_confusion_multiclase(
    matrix,
    ['positive', 'negative', 'neutral'],
    len(label_test)
)

Numero de datos para la prueba: 3836
Precision promedio: 0.769389967958389
Recall promedio: 0.7713147258501253

Matriz de confusion: 
+----------+------------+------------+-----------+
|          |   positive |   negative |   neutral |
+==========+============+============+===========+
| positive |        933 |         88 |       229 |
+----------+------------+------------+-----------+
| negative |         96 |       1068 |       168 |
+----------+------------+------------+-----------+
| neutral  |        178 |        123 |       953 |
+----------+------------+------------+-----------+

Metricas de desempeño: 
+----------+-------------+----------+
|          |   precision |   recall |
+==========+=============+==========+
| positive |    0.7464   | 0.772991 |
+----------+-------------+----------+
| negative |    0.801802 | 0.835027 |
+----------+-------------+----------+
| neutral  |    0.759968 | 0.705926 |
+----------+-------------+----------+



### Metricas obtenidas usando RandomForest con 300 modelos

In [75]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=300)

In [76]:
rfc.fit(data_train, label_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=300, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [77]:
predictions = rfc.predict(data_test)

In [80]:
matrix1 = confusion_matrix(label_test, predictions)
aciertos = sum([matrix[i][i] for i in range(matrix.shape[0])])
precision = aciertos / datos_totales_prueba
print("Numero de aciertos totales: " + str(aciertos))

Numero de aciertos totales: 2954


In [79]:
def imprimir_metricas_matriz_confusion_multiclase(matrix, headers, total_datos):
    print("Numero de datos para la prueba: " + str(total_datos))
    metrics = np.zeros([len(headers), 2])
    for i in range(matrix.shape[0]):
        precision = matrix[i][i] / sum(matrix[i])
        recoil = matrix[i][i] / sum([matrix[y][i] for y in range(matrix.shape[0])])
        metrics[i][0] = precision
        metrics[i][-1] = recoil
    precision_promedio = sum([metrics[y][0] for y in range(metrics.shape[0])]) / metrics.shape[0]
    recall_promedio = sum([metrics[y][-1] for y in range(metrics.shape[0])]) / metrics.shape[0]
    print("Precision promedio: " + str(precision_promedio))
    print("Recall promedio: " + str(recall_promedio))
    print("\nMatriz de confusion: \n" + str(tabulate(
        matrix,
        headers=headers,
        showindex=headers,
        tablefmt='grid')
    ))
    print("\nMetricas de desempeño: \n" + str(tabulate(
        metrics,
        headers=['precision', 'recall'],
        showindex=headers,
        tablefmt='grid')
    ))
    print("")


imprimir_metricas_matriz_confusion_multiclase(
    matrix1,
    ['positive', 'negative', 'neutral'],
    len(label_test)
)

Numero de datos para la prueba: 3836
Precision promedio: 0.814256599182915
Recall promedio: 0.8164783014679866

Matriz de confusion: 
+----------+------------+------------+-----------+
|          |   positive |   negative |   neutral |
+==========+============+============+===========+
| positive |        971 |         68 |       211 |
+----------+------------+------------+-----------+
| negative |         61 |       1142 |       129 |
+----------+------------+------------+-----------+
| neutral  |        134 |        106 |      1014 |
+----------+------------+------------+-----------+

Metricas de desempeño: 
+----------+-------------+----------+
|          |   precision |   recall |
+==========+=============+==========+
| positive |    0.7768   | 0.832762 |
+----------+-------------+----------+
| negative |    0.857357 | 0.867781 |
+----------+-------------+----------+
| neutral  |    0.808612 | 0.748892 |
+----------+-------------+----------+

